# Análisis de Serie Temporal - Consumo de Diésel

Este notebook documenta el análisis de la serie temporal del consumo de diésel, cubriendo las etapas requeridas en el laboratorio:

- Análisis exploratorio.
- Serie univariante.
- Descomposición y evaluación de componentes.
- Estacionariedad y transformación.
- Modelado ARIMA y Prophet.
- Comparación y selección del mejor modelo.

## Consolidación de Subcategorías de Diésel

Agrupamos las siguientes columnas:

- Diesel alto azufre
- Diesel bajo azufre
- Diesel ultra bajo azufre

En una sola columna llamada `Diesel`.

In [7]:
import pandas as pd

# Cargar datos
df_import = pd.read_csv('../data/importacion.csv', parse_dates=['Fecha'])
df_cons = pd.read_csv('../data/consumo.csv', parse_dates=['Fecha'])

# Combinar columnas de diésel
diesel_cols = ['Diesel alto azufre', 'Diesel bajo azufre', 'Diesel ultra bajo azufre']

# Verificar columnas
for col in diesel_cols:
    if col not in df_import.columns:
        print(f"Columna faltante en importacion: {col}")
    if col not in df_cons.columns:
        print(f"Columna faltante en consumo: {col}")

# Crear columna Diesel
df_import['Diesel'] = df_import[diesel_cols].sum(axis=1)
df_cons['Diesel'] = df_cons[diesel_cols].sum(axis=1)

# Filtrar columnas relevantes
cols = ['Fecha', 'Diesel']
df_import = df_import[cols].copy()
df_cons = df_cons[cols].copy()

# Revisar que esten bien
print(df_import.head())
print(df_cons.head()) 

       Fecha     Diesel
0 2001-01-01  566101.99
1 2001-02-01  489525.80
2 2001-03-01  575559.68
3 2001-04-01  437745.42
4 2001-05-01  552609.13
       Fecha     Diesel
0 2000-01-01  634667.06
1 2000-02-01  642380.66
2 2000-03-01  699807.25
3 2000-04-01  586803.98
4 2000-05-01  656948.20
